In [1]:
"""
  GitHubよりサンプルをダウンロード
"""
!git clone https://github.com/caribou68/ConvertMovieStyle.git
!mkdir ConvertMovieStyle/converted_images ConvertMovieStyle/diveded_images ConvertMovieStyle/out

Cloning into 'ConvertMovieStyle'...
remote: Enumerating objects: 18, done.
remote: Counting objects: 100% (18/18), done.
remote: Compressing objects: 100% (17/17), done.
remote: Total 18 (delta 3), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (18/18), 17.87 MiB | 20.04 MiB/s, done.
Resolving deltas: 100% (3/3), done.


#動画分解

In [2]:
!pip install ffmpeg-python
import ffmpeg

In [3]:
!ffmpeg -i "//content/ConvertMovieStyle/content_movie/sample.mp4" -progress - -r 60 "//content/ConvertMovieStyle/diveded_images/%06d.png"

ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

#画風変換

In [4]:
import os
import tensorflow as tf
# Load compressed models from tensorflow_hub
os.environ['TFHUB_MODEL_LOAD_FORMAT'] = 'COMPRESSED'

In [5]:
import numpy as np
import PIL.Image

!pip install tqdm
import tqdm

import tensorflow_hub as hub

In [6]:
def tensor_to_image(tensor):
  tensor = tensor*255
  tensor = np.array(tensor, dtype=np.uint8)
  if np.ndim(tensor)>3:
    assert tensor.shape[0] == 1
    tensor = tensor[0]
  return PIL.Image.fromarray(tensor)

In [7]:
def load_img(path_to_img):
  max_dim = 512
  img = tf.io.read_file(path_to_img)
  img = tf.image.decode_image(img, channels=3)
  img = tf.image.convert_image_dtype(img, tf.float32)

  shape = tf.cast(tf.shape(img)[:-1], tf.float32)
  long_dim = max(shape)
  scale = max_dim / long_dim

  new_shape = tf.cast(shape * scale, tf.int32)

  img = tf.image.resize(img, new_shape)
  img = img[tf.newaxis, :]
  return img

In [8]:
content_dir = '//content/ConvertMovieStyle/diveded_images/'
img_num = sum(os.path.isfile(os.path.join(content_dir,name)) for name in os.listdir(content_dir))
style_path = "//content/ConvertMovieStyle/style_images/sample1.jpg"
style_image = load_img(style_path)

In [9]:
for i in tqdm.tqdm(range(1, img_num)):
  content_path = content_dir + str(i).zfill(6) + ".png"

  content_image = load_img(content_path)

  hub_model = hub.load('https://tfhub.dev/google/magenta/arbitrary-image-stylization-v1-256/2')
  stylized_image = hub_model(tf.constant(content_image), tf.constant(style_image))[0]
  converted_img = tensor_to_image(stylized_image)
  converted_img.save("//content/ConvertMovieStyle/converted_images/" + str(i).zfill(6) + ".png")

100%|██████████| 900/900 [2:08:17<00:00,  8.55s/it]


#動画合成

In [10]:
!ffmpeg -framerate 60 -i "//content/ConvertMovieStyle/converted_images/%06d.png" "//content/ConvertMovieStyle/out/out.mp4" -y

ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

In [11]:
from IPython.display import HTML
from base64 import b64encode

video_file = open("//content/ConvertMovieStyle/out/out.mp4", "r+b").read()

video_url = f"data:video/mp4;base64,{b64encode(video_file).decode()}"
HTML(f"""<video width={600} controls><source src="{video_url}"></video>""")


Output hidden; open in https://colab.research.google.com to view.